#### Importando bibliotecas

In [8]:
import pandas as pd
import numpy as np

#### Importando os datasets de treinamento e teste

In [18]:
# Carregando os datasets
df_train_loaded = pd.read_csv("arquivos_csv/train_dataset1.csv")
df_test_loaded = pd.read_csv("arquivos_csv/test_dataset1.csv")

# Separando os dados de treinamento
X_train = df_train_loaded.drop("label", axis=1).values.T # (n_features, n_amostras)
y_train = df_train_loaded["label"].values.reshape(1, -1) # (1, n_amostras)

# Separando os dados de teste
X_test = df_test_loaded.drop("label", axis=1).values.T
y_test = df_test_loaded["label"].values.reshape(1, -1)